In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [2]:
power = pd.read_csv('lv2-2408.csv')

# 1

In [3]:
df1 = power.copy()

## 1-1

In [5]:
col = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6']

In [14]:
for c in col:
    q1 = df1.groupby(['SEASON', 'FAC'])[c].transform(lambda x: x.quantile(0.25))
    q2 = df1.groupby(['SEASON', 'FAC'])[c].transform(lambda x: x.quantile(0.5))
    q3 = df1.groupby(['SEASON', 'FAC'])[c].transform(lambda x: x.quantile(0.75))

    df1[f'{c}_outlier'] = df1[c] > q2 + 2 * (q3-q1)

In [20]:
df1['OUT'] = np.where(df1.iloc[:, -6:].sum(axis=1) >= 1, 1, 0)

## 1-2

In [23]:
df1.groupby(['SEASON', 'FAC'])['OUT'].mean().max().round(4)

0.3207

# 2

In [24]:
df2 = power.copy()

## 2-1

In [27]:
df2 = df2[df2['FAC'] == 'Northern']

## 2-2

In [32]:
df2['TOTAL'] = df2.iloc[:, -6:].sum(axis=1)

## 2-3
$$\bar{X} \pm {Z}_{{\alpha}/2} \frac{\sigma}{\sqrt{n}}$$

In [35]:
sigma = 220
z = 1.64

In [46]:
def get_interval(x):
    t1 = x.mean() + (z * (sigma / (x.shape[0]) ** 0.5))
    t2 = x.mean() - (z * (sigma / (x.shape[0]) ** 0.5))
    return t1 - t2

In [49]:
df2.groupby('SEASON').apply(lambda x: get_interval(x['TOTAL'])).max().round(1)

53.2

# 3

In [75]:
df3 = power.copy()

## 3-1

In [76]:
col = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6']

In [77]:
df3[col] = df3[col].replace({0: np.nan})

In [80]:
df3 = df3.dropna(subset=col).sort_values(by=['DATE', 'FAC'])

## 3-2

In [82]:
df3['F6_NEXT'] = df3.groupby('FAC')['F6'].transform(lambda x: x.shift(-1))

In [84]:
df3 = df3.dropna(subset=['F6_NEXT'])

In [85]:
df3.shape

(2775, 11)

## 3-3

In [89]:
df3['DATE'] = pd.to_datetime(df3['DATE'])

In [91]:
TRAIN3 = df3[df3['DATE'] < '2022-09-01']
TEST3 = df3[df3['DATE'] >= '2022-09-01']

## 3-4

In [92]:
from sklearn.linear_model import LinearRegression

In [93]:
model = LinearRegression().fit(X=TRAIN3[col], y=TRAIN3['F6_NEXT'])

## 3-5

In [95]:
pred = model.predict(X=TEST3[col])

In [103]:
round((100/len(pred)) * ((pred - TEST3['F6_NEXT']) / TEST3['F6_NEXT']).abs().sum(), 3)

8.639